In [1]:
import numpy as np
import matplotlib.pyplot as plt

import emcee
import corner

import _pickle as cPickle
import matplotlib as mpl

In [2]:
cat = np.genfromtxt('../forElena16Nov/SAGITTARIUS-II_cat16Nov.dat',names=True)

In [3]:
xgood = cat['x_f606w']
ygood = cat['y_f606w']
m606 = cat['magr_f606w']
dm606 = cat['err_f606w']
m814 = cat['magr_f814w']
dm814 = cat['err_f814w']

In [35]:
def median_interval(data, alpha=0.32):
    """
    Median including Bayesian credible interval.

    Parameters
    ----------
    data  : posterior samples
    alpha : 1 - confidence interval

    Returns
    -------
    [med,[lo, hi]] : median, lower, and upper percentiles
    
    """
    q = [100*alpha/2., 50, 100*(1-alpha/2.)]
    lo,med,hi = np.percentile(data,q)
    return [med,[lo,hi]]

In [4]:
# m606c = m606[dm606 < 0.025]
# dm606c = dm606[dm606 < 0.025]

# m814c = m814[dm606 < 0.025]
# dm814c = dm814[dm606 < 0.025]

m606c = m606
dm606c = dm606

m814c = m814
dm814c = dm814

In [5]:
def mag2flux(mag,ZPT=26.667):
    
    exp = ((-1 * mag) - ZPT) / (-2.5)
    
    flux = 10**exp
    
    return flux

In [27]:
fl_606 = mag2flux(m606c,ZPT=26.667)
df_606 = mag2flux(dm606c,ZPT=26.667)
fl_814 = mag2flux(m814c,ZPT=26.779)
df_814 = mag2flux(dm814c,ZPT=26.779)

In [28]:
perc_err_606 = df_606/fl_606
perc_err_814 = df_814/fl_814

avg_606 = np.average(perc_err_606)
avg_814 = np.average(perc_err_814)

In [30]:
w_606 = ((avg_606 - perc_err_606)/avg_606)**(-2)
w_814 = ((avg_814 - perc_err_814)/avg_814)**(-2)

In [36]:
samples_h = cPickle.load( open( "sag2mcmcPL1902-2.p", "rb" ) )
#MEDIAN VALUES AND +/-1 SIGMA FOR EACH FIT PARAMETER
rich_h,[rich_min,rich_max] = median_interval(samples_h[:,0])
x_h,[xmin,xmax] = median_interval(samples_h[:,1])
y_h,[ymin,ymax] = median_interval(samples_h[:,2])
sag2_ext_fit_h,[sag2_ext_min,sag2_ext_max] = median_interval(samples_h[:,3])
sag2_ell_fit_h,[sag2_ell_min,sag2_ell_max] = median_interval(samples_h[:,4])
sag2_pa_fit_h,[sag2_pa_min,sag2_pa_max] = median_interval(samples_h[:,5])

# res = [rich_h,rich_max-rich_h,rich_h-rich_min,x_h,xmax-x_h,x_h-xmin,y_h,ymax-y_h,y_h-ymin,
#            sag2_ext_fit_h,sag2_ext_max-sag2_ext_fit_h,sag2_ext_fit_h-sag2_ext_min,
#            sag2_ell_fit_h,sag2_ell_max-sag2_ell_fit_h,sag2_ell_fit_h-sag2_ell_min,
#            sag2_pa_fit_h,sag2_pa_max-sag2_pa_fit_h,sag2_pa_fit_h-sag2_pa_min]

# print(res)

In [37]:
gal_pa = sag2_pa_fit_h
x0 = x_h
y0 = y_h
gal_ell = sag2_ell_fit_h

In [38]:
costh = np.cos(-1 * np.deg2rad(gal_pa))
sinth = np.sin(-1 * np.deg2rad(gal_pa))

dx = xgood - x0
dy = ygood - y0

r1 = (dx * costh - dy * sinth) / (1 - gal_ell)
r2 = (dx * sinth + dy * costh)

radius = np.sqrt(r1**2 + r2**2)

In [41]:
sort_idx = np.argsort(radius)
sorted_rad = radius[sort_idx]
total_flux = fl_606.sum()
sorted_flux = fl_606[sort_idx]

In [42]:
part_flux = 0
ii = 0

while part_flux <= (0.5*total_flux):
    part_flux += sorted_flux[ii]
    ii += 1
    
idx_take = ii - 1
rad_half = radius[ii-1]

print(rad_half/1200)  # arcmin

1.6582688391223603


In [44]:
sat_val = 75000
flat_flux = fl_606
flat_flux[flat_flux > sat_val] = 75000
sorted_flux = flat_flux[sort_idx]
total_flux = flat_flux.sum()

In [45]:
part_flux = 0
ii = 0

while part_flux <= (0.5*total_flux):
    part_flux += sorted_flux[ii]
    ii += 1
    
idx_take = ii - 1
rad_half = radius[ii-1]

print(rad_half/1200)  # arcmin

1.1356912330639881
